# Results Notebook

In [436]:
%load_ext autoreload
%autoreload 2

import hwacctools.quantization.quant as q
from stim_lib.stimulus_gen import generate_qracc_inputs
import numpy as np
import plotly.express as px
import seaborn as sns
import pandas as pd
df = pd.DataFrame
sns.set_theme()

def melt_row_col_data(arr,name):
    arr_df = df(arr)
    arr_df = arr_df.melt(
        ignore_index=False,
        var_name='Column',
        value_name='ADC_OUT_AMS',
    )
    arr_df['Row'] = arr_df.index
    return arr_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [437]:
# Generate the reference result

w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 1,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)
wx = w @ x.T
wx_rounded = np.round(wx.T / 2**2).astype(int).T[::-1].T

[STIM_GEN] Generating random weights


### Comparing AMS vs RTL simulations of QRAcc

In [438]:
ams = np.loadtxt('../tb/qracc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/qracc/inputs/adc_out_rtlsims.txt')

ams_df = df(ams)
rtl_df = df(rtl)

# ams_df['Row'] = ams_df.index
# rtl_df['Row'] = rtl_df.index

ams_df = ams_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_AMS',
)
rtl_df = rtl_df.melt(
    ignore_index=False,
    var_name='Column',
    value_name='ADC_OUT_RTL',
)
ams_df['Row'] = ams_df.index
rtl_df['Row'] = rtl_df.index
# Join the two dataframes
adc_df = pd.merge(ams_df, rtl_df)

rtl_fp = rtl.flatten()/2**3
ams_fp = ams.flatten()/2**3

err =   rtl_fp - ams_fp
snr = 20*np.log(np.var(rtl_fp) / np.var(err))

rmse = np.sqrt(np.mean((rtl.flatten() - ams.flatten())**2))
fig = px.scatter(adc_df, y='ADC_OUT_AMS', x='ADC_OUT_RTL', width=400, height=400, title=f'AMS vs Shifted, RMSE={rmse:.2f}, SNR={snr:.2f}', labels={'ADC_OUT_AMS':'AMS','ADC_OUT_RTL':'RTL'})
fig.show()
fig.write_image('../images/rtl_vs_ams.svg')

/tmp/ipykernel_256777/2330861704.py:29: RuntimeWarning:

divide by zero encountered in scalar divide



# Sequential Accelerator Simulations

In [439]:
w,x,wx_outBits = generate_qracc_inputs(
    wDimX = 32, #nColumns
    wDimY = 128, #nRows
    xBatches = 10,
    xTrits = 3,
    outBits = 4,
    seed = 0,
    weight_mode = 'bipolar',
    col_symmetric=False
)

# ams = np.loadtxt('../tb/seq_acc/inputs/adc_out.txt')
rtl = np.loadtxt('../tb/seq_acc/inputs/mac_out_rtlsims.txt')

wx = w @ x.T

# rtl.astype(int), wx[::-1].T

rtl_fp = rtl.flatten()/(2**8)
wx_fp = wx[::-1].T.flatten()/(2**8)

rmse = np.sqrt(np.mean((rtl_fp - wx_fp)**2))
# SNR

err = rtl_fp - wx_fp
snr = 20*np.log(np.var(rtl_fp) / np.var(err))

fig = px.scatter(
    x=rtl.flatten(), 
    y=wx[::-1].T.flatten(), 
    width=400, 
    height=400, 
    title=f'Ideal vs RTL, RMSE={rmse:.2f}, SNR={snr:.2f}', 
    labels={'x': 'RTL', 'y': 'Ideal'}
)
fig.show()

[STIM_GEN] Generating random weights


In [440]:
q.simulate_trit_mul(w, x[0], trits = 3, verbose=True)

x trits:
     0  1  2
0    0 -1 -1
1    1  0  0
2    0 -1  0
3    0  1  0
4   -1  0  0
..  .. .. ..
123 -1 -1  0
124 -1  0  0
125 -1 -1 -1
126 -1  0  0
127 -1 -1  0

[128 rows x 3 columns]
partials:
   0   1   2   3   4   5   6   7   8   9   ...  22  23  24  25  26  27  28  \
0  -3  13  -1  -5   3  -5  -5   1 -11   9  ...  -7  -9   1   7   5   7  -1   
1 -11   3   3   7   5  -3   3  -7  -5  11  ...  11   7   5  -3   1  11 -11   
2  -4  -4  -6   0   8   2   6  -8  -6  10  ... -10   0   8  14   0   8   2   

   29  30  31  
0   9 -21   3  
1   9 -15   1  
2   6  -4   8  

[3 rows x 32 columns]
shifting accumulator:
   0   1   2   3   4   5   6   7   8   9   ...  22  23  24  25  26  27  28  \
0 -16 -16 -24   0  32   8  24 -32 -24  40  ... -40   0  32  56   0  32   8   
1 -52   4   0  28  36  -8  24 -44 -32  64  ...  24  28  36  16   4  60 -40   
2 -38  54  -4  -6  30 -24  -8 -18 -60  68  ... -16 -22  22  36  22  58 -24   

   29   30  31  
0  24  -16  32  
1  48  -68  20  
2  60 -118  22 

array([  22., -118.,   60.,  -24.,   58.,   22.,   36.,   22.,  -22.,
        -16.,  -40.,  -28.,  -58.,   24.,  -16.,  -44.,   76.,    4.,
         46.,  -78.,   -2.,  -66.,   68.,  -60.,  -18.,   -8.,  -24.,
         30.,   -6.,   -4.,   54.,  -38.])

In [441]:
x[1] @ w.T

array([112,   8,  28,   8, -64,  36, -40,  60, -50,  22,   8,  14, -40,
         2,  30,  42,  12, -18, -18,  42,  72, -64, -46, -46,  30,  42,
        36, -16, -50,  26,  14, -28])

In [442]:
df(rtl.astype(int)) 

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-52,56,-10,-12,30,-30,-14,-24,-82,98,...,-20,-52,22,30,16,66,-38,90,-100,22
1,-34,8,20,-92,-22,42,42,24,-76,-76,...,16,-76,54,-46,34,-70,-4,34,2,98
2,46,-112,66,26,-50,-64,-32,78,24,14,...,36,-36,-74,98,-58,90,36,2,12,78
3,88,-38,-28,6,32,54,-20,-102,26,4,...,-56,-112,-68,56,-80,-10,98,-20,10,-4
4,-34,-64,-80,-30,-14,66,22,-2,-32,-52,...,42,0,-72,-60,-18,-44,98,50,42,30
5,12,-16,-30,-16,-58,20,-24,18,98,-56,...,-46,20,50,-32,-22,-112,-66,4,2,-66
6,84,-56,-14,-24,-64,-52,-22,-30,6,-6,...,-2,-32,38,-88,52,12,12,50,22,90
7,50,-112,-84,22,-80,20,-104,98,48,12,...,40,-96,-20,-14,56,-26,-16,46,-88,26
8,98,0,-72,-64,58,-76,6,22,-32,82,...,-56,50,70,-112,-24,-56,34,66,-98,30
9,-24,-84,14,-16,-82,-2,58,-80,-98,-56,...,98,-98,20,-98,80,66,72,-2,-48,70


In [443]:
df(wx[::-1].T)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-38,54,-4,-6,30,-24,-8,-18,-60,68,...,-16,-22,22,36,22,58,-24,60,-118,22
1,-28,14,26,-50,-16,36,42,30,-46,-46,...,22,-50,60,-40,36,-64,8,28,8,112
2,48,-80,48,24,-40,-34,-26,96,10,20,...,42,-26,-60,70,-56,92,42,8,18,54
3,61,-29,-27,7,33,55,-19,-77,25,5,...,-51,-83,-43,41,-79,-9,77,-13,11,3
4,-28,-52,-78,-24,-8,76,28,4,-26,-46,...,34,6,-90,-54,-12,-36,102,32,36,36
5,17,-11,-21,-11,-53,25,-19,19,147,-37,...,-33,13,39,-21,-17,-65,-61,15,3,-53
6,69,-49,-9,-19,-35,-35,-17,-25,11,-1,...,3,-27,43,-91,53,17,17,37,33,83
7,47,-103,-79,27,-57,25,-83,113,45,5,...,45,-77,-15,-9,53,-25,-11,35,-69,13
8,142,4,-68,-64,54,-64,10,22,-28,74,...,-48,30,66,-88,-20,-46,22,54,-74,28
9,-17,-53,21,-9,-51,5,57,-43,-55,-37,...,105,-87,27,-71,75,69,63,5,-41,69
